In [1]:

# coding: utf-8
from __future__ import print_function
import os
import numpy as np
np.random.seed(1337)

import pandas as pd
import theano
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Flatten
from keras.layers import Convolution1D, MaxPooling1D, Embedding, LSTM
from keras.models import Model
from keras.layers import Input, Dropout
from keras.optimizers import SGD, Adadelta
from keras.models import Sequential
import sys

databaseConnectionServer = 'srn02.cs.cityu.edu.hk'
documentTable = 'document'

BASE_DIR = '../../'
GLOVE_DIR = BASE_DIR + 'glove/'

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.3
CONVOLUTION_FEATURE = 256
BORDER_MODE = 'valid'
DENSE_FEATURE = 1024
DROP_OUT = 0.3
LEARNING_RATE=0.0001
MOMENTUM=0.9
EPOCH=25
BATCH_SIZE=128
embed = 100
embedfile = 'glove.6B.50d.txt'
authorList = [21,63,27,88] 
doc_id = 544
chunk_size = 1000
nb_epoch = 30
EPOCH = nb_epoch

Using TensorFlow backend.


In [ ]:
# first, build index mapping words in the embeddings set
# to their embedding vector

print('Indexing word vectors.')

embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

# second, prepare text samples and their labels
print('Processing text dataset')

Indexing word vectors.
Found 400000 word vectors.
Processing text dataset


In [ ]:
texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
import DatabaseQuery
# textToUse = pd.read_csv("suffle_4_6000.csv", names=["author_id", "doc_content"], dtype={'author_id': int})
from sshtunnel import SSHTunnelForwarder
PORT=5432
with SSHTunnelForwarder(('srn02.cs.cityu.edu.hk', 22),
                        ssh_username='stylometry',
                        ssh_password='stylometry',
                        remote_bind_address=('localhost', 5432),
                        local_bind_address=('localhost', 5400)):
    textToUse = DatabaseQuery.getWordGenderData(5400, doc_id, documentTable = documentTable, chunk_size = chunk_size)

Execution completed
Read completed
Number of rows: 14
Execution completed
Read completed
Number of rows: 14
Execution completed
Read completed
Number of rows: 2385
{1: 'M', 2: 'M', 3: 'M', 4: 'M', 5: 'M', 6: 'M', 7: 'M', 8: 'M', 9: 'M', 10: 'M', 11: 'M', 12: 'M', 13: 'M', 14: 'M', 15: 'M', 16: 'M', 17: 'M', 18: 'M', 19: 'M', 20: 'M', 21: 'F', 22: 'M', 23: 'M', 24: 'F', 25: 'M', 26: 'M', 27: 'M', 28: 'M', 29: 'M', 30: 'M', 31: 'M', 32: 'M', 33: 'M', 34: 'M', 35: 'M', 36: 'F', 37: 'M', 38: 'M', 39: 'M', 40: 'F', 41: 'M', 42: 'M', 43: 'M', 44: 'M', 45: 'M', 46: 'M', 47: 'F', 48: 'M', 49: 'F', 50: 'M', 51: 'M', 52: 'M', 53: 'M', 54: 'M', 55: 'M', 56: 'M', 57: 'M', 58: 'M', 59: 'M', 60: 'M', 61: 'M', 62: 'M', 63: 'M', 64: 'M', 65: 'M', 66: 'M', 67: 'M', 68: 'M', 69: 'M', 70: 'M', 71: 'M', 72: 'M', 73: 'F', 74: 'M', 75: 'M', 76: 'F', 77: 'M', 78: 'F', 79: 'M', 80: 'M', 81: 'M', 82: 'F', 83: 'M', 84: 'M', 85: 'M', 86: 'M', 87: 'F', 88: 'M', 89: 'M', 90: 'M', 91: 'M', 92: 'M', 93: 'M', 94: 'M'

In [ ]:
labels = []
texts = []
size = []
genderList = textToUse.gender.unique()

In [ ]:
genderList

In [ ]:
for auth in authorList:
    current = textToUse.loc[textToUse['author_id'] == auth]
    size.append(current.shape[0])
    print("Author: %5s  Size: %5s" % (auth, max(size)))
print("Min: %s" % (min(size)))
print("Max: %s" % (max(size)))

authorList = authorList.tolist()

In [ ]:
labels = []
texts = []
maxRows = min(size)
for auth in authorList:
    current = textToUse.loc[textToUse['author_id'] == auth]
    current = current.sample(n = maxRows)
    textlist = current.doc_content.tolist()
    texts = texts + textlist
    labels = labels + [authorList.index(author_id) for author_id in current.author_id.tolist()]
labels_index = {}
labels_index[0] = 0
for i, auth in enumerate(authorList):
    labels_index[i] = auth

del textToUse

print('Found %s texts.' % len(texts))
print('Found %s labels.' % len(labels))

In [ ]:
# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=VALIDATION_SPLIT)


In [ ]:
print(y_train[4])
print(len(x_train[4]))
print(type(x_train[4]))

In [ ]:
print('Preparing embedding matrix.')

# prepare embedding matrix
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

print('Training model.')

In [ ]:
print(y_train[4])
print(len(x_train[4]))

In [ ]:
print(nb_words)
print(MAX_NB_WORDS)
print(len(word_index))

In [ ]:

model = Sequential()

model.add(Embedding(                          # Layer 0, Start
    input_dim=nb_words + 1,                   # Size to dictionary, has to be input + 1
    output_dim=EMBEDDING_DIM,                 # Dimensions to generate
    weights=[embedding_matrix],               # Initialize word weights
    input_length=MAX_SEQUENCE_LENGTH))        # Define length to input sequences in the first layer

model.add(Convolution1D(                      # Layer 1,   Features: 256, Kernel Size: 7
    nb_filter=CONVOLUTION_FEATURE,            # Number of kernels or number of filters to generate
    filter_length=7,                          # Size of kernels
    border_mode=BORDER_MODE,                  # Border = 'valid', cause kernel to reduce dimensions
    activation='relu'))                       # Activation function to use

model.add(MaxPooling1D(                       # Layer 1a,  Max Pooling: 3
    pool_length=3))                           # Size of kernels

model.add(Convolution1D(                      # Layer 2,   Features: 256, Kernel Size: 7
    nb_filter=CONVOLUTION_FEATURE,            # Number of kernels or number of filters to generate
    filter_length=7,                          # Size of kernels
    border_mode=BORDER_MODE,                  # Border = 'valid', cause kernel to reduce dimensions
    activation='relu'))                       # Activation function to use

model.add(MaxPooling1D(                       # Layer 2a,  Max Pooling: 3
    pool_length=3))                           # Size of kernels

model.add(Convolution1D(                      # Layer 3,   Features: 256, Kernel Size: 3
    nb_filter=CONVOLUTION_FEATURE,            # Number of kernels or number of filters to generate
    filter_length=3,                          # Size of kernels
    border_mode=BORDER_MODE,                  # Border = 'valid', cause kernel to reduce dimensions
    activation='relu'))                       # Activation function to use

model.add(Convolution1D(                      # Layer 4,   Features: 256, Kernel Size: 3
    nb_filter=CONVOLUTION_FEATURE,            # Number of kernels or number of filters to generate
    filter_length=3,                          # Size of kernels
    border_mode=BORDER_MODE,                  # Border = 'valid', cause kernel to reduce dimensions
    activation='relu'))                       # Activation function to use

model.add(Convolution1D(                      # Layer 5,   Features: 256, Kernel Size: 3
    nb_filter=CONVOLUTION_FEATURE,            # Number of kernels or number of filters to generate
    filter_length=3,                          # Size of kernels
    border_mode=BORDER_MODE,                  # Border = 'valid', cause kernel to reduce dimensions
    activation='relu'))                       # Activation function to use

model.add(Convolution1D(                      # Layer 6,   Features: 256, Kernel Size: 3
    nb_filter=CONVOLUTION_FEATURE,            # Number of kernels or number of filters to generate
    filter_length=5,                          # Size of kernels
    border_mode=BORDER_MODE,                  # Border = 'valid', cause kernel to reduce dimensions
    activation='relu'))                       # Activation function to use

model.add(MaxPooling1D(                       # Layer 6a,  Max Pooling: 3
    pool_length=3))                           # Size of kernels

model.add(Flatten())                          # Layer 7

model.add(Dense(                              # Layer 7a,  Output Size: 1024
    output_dim=DENSE_FEATURE,                 # Output dimension
    activation='relu'))                       # Activation function to use

model.add(Dropout(DROP_OUT))

model.add(Dense(                              # Layer 8,   Output Size: 1024
    output_dim=DENSE_FEATURE,                 # Output dimension
    activation='relu'))                       # Activation function to use

model.add(Dropout(DROP_OUT))

model.add(Dense(                              # Layer 9,  Output Size: Size Unique Labels, Final
    output_dim=len(labels_index),             # Output dimension
    activation='softmax'))                    # Activation function to use

# model = Model(start, end)

sgd = SGD(lr=LEARNING_RATE, momentum=MOMENTUM, nesterov=True)

adadelta = Adadelta(lr=1.0, rho=0.95, epsilon=1e-08)

model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])

print("Done compiling.")

In [ ]:
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=EPOCH, batch_size=BATCH_SIZE)


In [ ]:
texts = []  # list of text samples
labels = []  # list of label ids
import DatabaseQuery
# textToUse = pd.read_csv("suffle_4_6000.csv", names=["author_id", "doc_content"], dtype={'author_id': int})
from sshtunnel import SSHTunnelForwarder
PORT=5432
with SSHTunnelForwarder(('srn02.cs.cityu.edu.hk', 22),
                        ssh_username='stylometry',
                        ssh_password='stylometry',
                        remote_bind_address=('localhost', 5432),
                        local_bind_address=('localhost', 5400)):
    textToUse = DatabaseQuery.getWordDocData(5400, doc_id, chunk_size = chunk_size)
labels = []
texts = []
for index, row in textToUse.iterrows():
    labels.append(authorList.index(row.author_id))
    texts.append(row.doc_content)
        
print('Found %s texts.' % len(texts))

del textToUse


In [ ]:
# finally, vectorize the text samples into a 2D integer tensor
sequences = tokenizer.texts_to_sequences(texts)

print('Found %s unique tokens.' % len(word_index))

X = pad_sequences(sequences, maxlen = chunk_size)

print('Shape of data tensor:', X.shape)

testX = X[:]


In [ ]:
# Function to take input of data and return prediction model
predY = np.array(model.predict(testX, batch_size=128))
def entroPred(predY):
    predYList = predY[:]
    entro = []
    import math
    for row in predY:
        entroval = 0
        for i in row:
            entroval += (i * (math.log(i , 2)))
        entroval = -1 * entroval
        entro.append(entroval)
    yx = zip(entro, predY)
    yx = sorted(yx, key = lambda t: t[0])
    newPredY = [x for y, x in yx]
    predYEntroList = newPredY[:int(len(newPredY)*0.9)]
    predY = np.mean(predYEntroList, axis=0)
    return predY

In [ ]:
print(labels_index)

In [ ]:
print(entroPred(predY[:]))

In [ ]:
print(predY)